In [1]:
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd
import numpy as np
from mordred import Calculator, descriptors
from multiprocessing import freeze_support
from tqdm import tqdm
import warnings
import gc
warnings.filterwarnings("ignore")

In [2]:
dbslim_df = pd.read_csv('drugbank-slim.csv')

In [3]:
dbslim_df.shape

(2420, 32)

In [4]:
dbslim_df.head()

,type,drugbank_id,name,description,pharmacodynamics,mechanism-of-action,toxicity,metabolism,absorption,half-life,...,logP,Water_Solubility,Melting_Point,Boiling_Point,Hydrophobicity,Isoelectric_Point,caco2_Permeability,pKa,Radioactivity,aliases
0,small molecule,DB00006,Bivalirudin,"Bivalirudin is a synthetic 20 residue peptide (thrombin inhibitor) which reversibly inhibits thrombin. Once bound to the active site, thrombin cannot activate fibrinogen into fibrin, the crucial step in the formation of thrombus. It is administered intravenously. Because it can cause blood stagnation, it is important to monitor changes in hematocrit, activated partial thromboplastin time, international normalized ratio and blood pressure.",Bivalirudin directly and reversibly inhibits thrombin by specifically binding both to the catalytic site and to the anion-binding exosite of circulating and clot-bound thrombin. The action of bivalirudin is reversible because thrombin will slowly cleave the thrombin-bivalirudin bond which recovers the active site of thrombin.,"Inhibits the action of thrombin by binding both to its catalytic site and to its anion-binding exosite. Thrombin is a serine proteinase that plays a central role in the thrombotic process, acting to cleave fibrinogen into fibrin monomers and to activate Factor XIII to Factor XIIIa, allowing fibrin to develop a covalently cross-linked framework which stabilizes the thrombus; thrombin also activates Factors V and VIII, promoting further thrombin generation, and activates platelets, stimulating aggregation and granule release.","Based on a study by Gleason et al., the no-observed-adverse-effect level (NOAEL) for bivalirudin, administered to rats via intravenous infusion over a 24-hour period, was 2000 mg/kg/24 h.",80% proteolytic cleavage,"Following intravenous administration, bivalirudin exhibits linear pharmacokinetics. The mean steady state concentration is 12.3 +/- 1.7mcg/mL after administration of an intravenous bolus of 1mg/kg followd by a 2.5mg/kg/hr intravenous infusion given over 4 hours.",* Normal renal function: 25 min (in normal conditions)\r\n* Creatinine clearance 10-29mL/min: 57min\r\n* Dialysis-dependant patients: 3.5h,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n |Angiomax|Bivalirudin|Bivalirudin for Injection|Bivalirudin in 0.9% Sodium Chloride
1,small molecule,DB00014,Goserelin,"Goserelin is a synthetic hormone. In men, it stops the production of the hormone testosterone, which may stimulate the growth of cancer cells. In women, goserelin decreases the production of the hormone estradiol (which may stimulate the growth of cancer cells) to levels similar to a postmenopausal state. When the medication is stopped, hormone levels return to normal.","The pharmacokinetics of goserelin have been determined in both male and female healthy volunteers and patients. In these studies, goserelin was administered as a single 250µg (aqueous solution) dose and as a single or multiple 3.6 mg depot dose by subcutaneous route.",Goserelin is a synthetic decapeptide analogue of LHRH. Goserelin acts as a potent inhibitor of pituitary gonadotropin secretion when administered in the biodegradable formulation. The result is sustained suppression of LH and serum testosterone levels.,No experience of overdosage from clinical trials.,Hepatic,"Inactive orally, rapidly absorbed following subcutaneous administration.",4-5 hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n |Goserelin|Zoladex|Zoladex LA
2,small molecule,DB00027,Gramicidin D,"Gramcidin D is a heterogeneous mixture of three antibiotic compounds, gramicidins A, B and C, making up 80%, 6%, and 14% respectively all of which are obtained from the soil bacterial species Bacillus brevis and called collectively gramicidin D. Gramcidins are 15 residue peptides with alternating D and L amino acids. The peptides assemble inside of the hydrophobic interior of the cellular lipid bilayer to form a β-helix. The helix itself is not long enough to span the membran

In [6]:
dbsmiles_df = pd.read_csv('drugbank_smiles_df.csv')

In [7]:
dbsmiles_df.shape

(7551, 16)

In [8]:
dbsmiles_df.head()

,drugbank_id,name,cas,smiles,logP ALOGPS,logP ChemAxon,solubility ALOGPS,pKa (strongest acidic),pKa (strongest basic),F,carbonyl,sulfinyl,sulfonyl,nitroso,nitro,tertAmines
0,DB00006,Bivalirudin,128270-60-0,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)[C@H](CC(O)=O)NC(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@@H]1CCCN1C(=O)[C@H](N)CC1=CC=CC=C1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC(C)C)C(O)=O,-0.76,-14.0,4.64e-02 g/l,2.79,11.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DB00014,Goserelin,65807-02-5,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@H](CC1=CNC2=CC=CC=C12)NC(=O)[C@H](CC1=CN=CN1)NC(=O)[C@@H]1CCC(=O)N1)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N1CCC[C@H]1C(=O)NNC(N)=O,0.30,-5.2,2.83e-02 g/l,9.27,10.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DB00035,Desmopressin,16679-58-6,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)[C@H](CC2=CC=C(O)C=C2)NC(=O)CCSSC[C@H](NC(=O)[C@H](CC(N)=O)NC1=O)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCCNC(N)=N)C(=O)NCC(N)=O,-1.00,-6.1,1.10e-01 g/l,9.50,11.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DB00050,Cetrorelix,120287-85-6,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@@H](CC1=CN=CC=C1)NC(=O)[C@@H](CC1=CC=C(Cl)C=C1)NC(=O)[C@@H](CC1=CC2=CC=CC=C2C=C1)NC(C)=O)C(=O)N[C@@H](CCCNC(N)=N)C(=O)N1CCC[C@H]1C(=O)N[C@H](C)C(N)=O,1.33,-1.7,6.94e-03 g/l,9.49,11.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DB00080,Daptomycin,103060-53-3,CCCCCCCCCC(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CC(O)=O)C(=O)N[C@H]1[C@@H](C)OC(=O)[C@H](CC(=O)C2=C(N)C=CC=C2)NC(=O)[C@@H](NC(=O)[C@@H](CO)NC(=O)CNC(=O)[C@H](CC(O)=O)NC(=O)[C@@H](C)NC(=O)[C@H](CC(O)=O)NC(=O)[C@H](CCCN)NC(=O)CNC1=O)[C@H](C)CC(O)=O,-0.47,-9.4,1.73e-02 g/l,2.98,9.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for j in tqdm(range(len(dbslim_df))):
    
    for i in range(len(dbsmiles_df)):
        if dbsmiles_df.loc[i,'drugbank_id'] == dbslim_df.loc[j,'drugbank_id']:
            dbslim_df.loc[j,'smiles'] = dbsmiles_df.loc[i,'smiles']

100%|██████████████████████████████████████████████████████████████████████████████| 2420/2420 [07:20<00:00,  5.60it/s]


In [18]:
dbslim_df.head()

,type,drugbank_id,name,description,pharmacodynamics,mechanism-of-action,toxicity,metabolism,absorption,half-life,...,Water_Solubility,Melting_Point,Boiling_Point,Hydrophobicity,Isoelectric_Point,caco2_Permeability,pKa,Radioactivity,aliases,smiles
0,small molecule,DB00006,Bivalirudin,"Bivalirudin is a synthetic 20 residue peptide (thrombin inhibitor) which reversibly inhibits thrombin. Once bound to the active site, thrombin cannot activate fibrinogen into fibrin, the crucial step in the formation of thrombus. It is administered intravenously. Because it can cause blood stagnation, it is important to monitor changes in hematocrit, activated partial thromboplastin time, international normalized ratio and blood pressure.",Bivalirudin directly and reversibly inhibits thrombin by specifically binding both to the catalytic site and to the anion-binding exosite of circulating and clot-bound thrombin. The action of bivalirudin is reversible because thrombin will slowly cleave the thrombin-bivalirudin bond which recovers the active site of thrombin.,"Inhibits the action of thrombin by binding both to its catalytic site and to its anion-binding exosite. Thrombin is a serine proteinase that plays a central role in the thrombotic process, acting to cleave fibrinogen into fibrin monomers and to activate Factor XIII to Factor XIIIa, allowing fibrin to develop a covalently cross-linked framework which stabilizes the thrombus; thrombin also activates Factors V and VIII, promoting further thrombin generation, and activates platelets, stimulating aggregation and granule release.","Based on a study by Gleason et al., the no-observed-adverse-effect level (NOAEL) for bivalirudin, administered to rats via intravenous infusion over a 24-hour period, was 2000 mg/kg/24 h.",80% proteolytic cleavage,"Following intravenous administration, bivalirudin exhibits linear pharmacokinetics. The mean steady state concentration is 12.3 +/- 1.7mcg/mL after administration of an intravenous bolus of 1mg/kg followd by a 2.5mg/kg/hr intravenous infusion given over 4 hours.",* Normal renal function: 25 min (in normal conditions)\r\n* Creatinine clearance 10-29mL/min: 57min\r\n* Dialysis-dependant patients: 3.5h,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n |Angiomax|Bivalirudin|Bivalirudin for Injection|Bivalirudin in 0.9% Sodium Chloride,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)[C@H](CC(O)=O)NC(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@@H]1CCCN1C(=O)[C@H](N)CC1=CC=CC=C1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC(C)C)C(O)=O
1,small molecule,DB00014,Goserelin,"Goserelin is a synthetic hormone. In men, it stops the production of the hormone testosterone, which may stimulate the growth of cancer cells. In women, goserelin decreases the production of the hormone estradiol (which may stimulate the growth of cancer cells) to levels similar to a postmenopausal state. When the medication is stopped, hormone levels return to normal.","The pharmacokinetics of goserelin have been determined in both male and female healthy volunteers and patients. In these studies, goserelin was administered as a single 250µg (aqueous solution) dose and as a single or multiple 3.6 mg depot dose by subcutaneous route.",Goserelin is a synthetic decapeptide analogue of LHRH. Goserelin acts as a potent inhibitor of pituitary gonadotropin secretion when administered in the biodegradable formulation. The result is sustained suppression of LH and serum testosterone levels.,No experience of overdosage from clinical trials.,Hepatic,"Inactive orally, rapidly absorbed following subcutaneous administration.",4-5 hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n |Goserelin|Zoladex|Zoladex LA,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@H](CC1=CNC2=CC=CC=C12)NC(=O)[C@H](CC1=CN=CN1)NC(=O)[C@@H]1CCC(=O)N1)C(=O)N[C@@H

In [20]:
dbslim_df.shape

(2420, 33)

In [21]:
dbslim_df = dbslim_df.dropna(subset=['smiles'])
dbslim_df.shape

(1696, 33)

In [23]:
dbslim_df = dbslim_df.dropna(subset=['description'])
dbslim_df.shape

(1619, 33)

In [24]:
dbslim_df.to_csv('dbslim_df_revised.csv', index=False)

In [26]:
calc = Calculator(descriptors, ignore_3D=False)

In [27]:
len(calc.descriptors)

1825

In [28]:
mols = [Chem.MolFromSmiles(smi) for smi in dbslim_df['smiles']]

In [29]:
dbslim_df['mol'] = pd.Series(mols)
dbslim_df = dbslim_df.dropna(subset=['mol'])

In [30]:
mols = dbslim_df['mol'].values

In [31]:
dbslim_df = dbslim_df.drop(['mol'], axis=1)

In [32]:
dbslim_df.to_csv('dbslim_df_revised.csv', index=False)

In [33]:
df = calc.pandas(mols)
filename = 'dbslim_descriptors_df'+str(i)+'.csv'
df.to_csv(filename, index=False)
del df
gc.collect()

C:\Users\elahi\Anaconda3\envs\gui\lib\site-packages\numpy\core\fromnumeric.py:1897: RuntimeWarning: overflow encountered in reduce
  return um.multiply.reduce(a, axis=axis, dtype=dtype, out=out, **kwargs)


C:\Users\elahi\Anaconda3\envs\gui\lib\site-packages\numpy\core\fromnumeric.py:1897: RuntimeWarning: overflow encountered in reduce
  return um.multiply.reduce(a, axis=axis, dtype=dtype, out=out, **kwargs)


C:\Users\elahi\Anaconda3\envs\gui\lib\site-packages\numpy\core\fromnumeric.py:1897: RuntimeWarning: overflow encountered in reduce
  return um.multiply.reduce(a, axis=axis, dtype=dtype, out=out, **kwargs)


C:\Users\elahi\Anaconda3\envs\gui\lib\site-packages\numpy\core\fromnumeric.py:1897: RuntimeWarning: overflow encountered in reduce
  return um.multiply.reduce(a, axis=axis, dtype=dtype, out=out, **kwargs)


100%|██████████████████████████████████████████████████████████████████████████████| 1242/1242 [03:23<00:00,  6.12it/s]


0